# Imports

In [1]:
!pip install -U transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import wandb
import json
import os

In [3]:
wandb.login(key="16a767377715590d2d5fe6351174577f96db6dc6")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

# Variables

In [4]:
LABEL_COLS = ["anger", "joy", "fear", "surprise", "sadness"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
WANDB_PROJECT = "23f3001910-t32025"
WANDB_ENTITY = "23f3001910-indian-institute-of-technology-madras"

DEBERTA_ARTIFACTS = [
    "deberta_large_seed_42:v1",
    "deberta_large_seed_2025:v0",
    "deberta_large_seed_123:v0",
]

ROBERTA_ARTIFACTS = [
    "deberta_large_seed_42:v2",
    "deberta_large_seed_2025:v1",
    "deberta_large_seed_123:v1",
]

In [6]:
DEBERTA_WEIGHT = 0.45
ROBERTA_WEIGHT = 0.55

In [7]:
VAL_PATH = "/kaggle/input/mydatasetfordl/augmented_train.csv"
val_df = pd.read_csv(VAL_PATH)



TEST_PATH = "/kaggle/input/mydatasetfordl/test_clean.csv"
test = pd.read_csv(TEST_PATH)
test_texts = test["text"].astype(str).tolist()

# Download Artifacts

In [8]:
def download_wandb_artifacts(artifact_names, model_type="DeBERTa"):

    local_paths = []


    run = wandb.init(
        project=WANDB_PROJECT,
        entity=WANDB_ENTITY,
        job_type="inference",
        name=f"ensemble_{model_type.lower()}",
    )

    for idx, artifact_name in enumerate(artifact_names):
        try:

            artifact = run.use_artifact(artifact_name, type='model')
            artifact_dir = artifact.download()

            local_paths.append(artifact_dir)

        except Exception as e:

            try:
                artifact_name_no_version = artifact_name.split(':')[0]
                artifact = run.use_artifact(f"{artifact_name_no_version}:latest", type='model')
                artifact_dir = artifact.download()
                local_paths.append(artifact_dir)
            except Exception as e2:

                continue

    wandb.finish()
    return local_paths

# Thresholds

In [9]:
def find_optimal_thresholds(val_probs, val_labels):
    thresholds = {}


    for i, label in enumerate(LABEL_COLS):
        best_f1 = 0
        best_thresh = 0.5

        for thresh in np.arange(0.2, 0.8, 0.05):
            preds = (val_probs[:, i] >= thresh).astype(int)
            f1 = f1_score(val_labels[:, i], preds, zero_division=0)

            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        thresholds[label] = best_thresh
        print(f"{label:10s}: threshold={best_thresh:.2f}, F1={best_f1:.4f}")


    return thresholds

# Load Models

In [10]:
def find_model_files(artifact_dir):
    if os.path.exists(os.path.join(artifact_dir, "config.json")):
        return artifact_dir

    for item in os.listdir(artifact_dir):
        subdir = os.path.join(artifact_dir, item)
        if os.path.isdir(subdir):
            if os.path.exists(os.path.join(subdir, "config.json")):
                return subdir

    return artifact_dir

#Predictions

In [11]:
def get_model_predictions(model_paths, texts, model_type="DeBERTa"):

    all_preds = []


    print(f"Generating {model_type} Predictions")


    for idx, model_path in enumerate(model_paths):

        try:
            actual_model_path = find_model_files(model_path)


            tokenizer = AutoTokenizer.from_pretrained(
                actual_model_path,
                local_files_only=True,
                trust_remote_code=True
            )
            model = AutoModelForSequenceClassification.from_pretrained(
                actual_model_path,
                local_files_only=True,
                trust_remote_code=True
            )
            model.to(DEVICE)
            model.eval()

            batch_preds = []
            BATCH_SIZE = 16


            for i in range(0, len(texts), BATCH_SIZE):
                batch_texts = texts[i:i+BATCH_SIZE]
                inputs = tokenizer(
                    batch_texts,
                    return_tensors="pt",
                    truncation=True,
                    padding=True,
                    max_length=512
                ).to(DEVICE)

                with torch.no_grad():
                    outputs = model(**inputs)
                    probs = torch.sigmoid(outputs.logits).cpu().numpy()
                    batch_preds.extend(probs)

                if (i // BATCH_SIZE + 1) % 10 == 0:
                    print(f"    Processed {i + len(batch_texts)}/{len(texts)} texts")

            all_preds.append(np.array(batch_preds))

            # Free memory
            del model
            torch.cuda.empty_cache()

        except Exception as e:
            import traceback
            traceback.print_exc()
            continue

    if len(all_preds) == 0:
        raise ValueError(f"No {model_type} models were successfully loaded!")

    avg_preds = np.mean(all_preds, axis=0)

    return avg_preds

# INFERENCE

In [12]:

from sklearn.model_selection import train_test_split
_, temp_df = train_test_split(val_df, test_size=0.2, random_state=42)
val_df, _ = train_test_split(temp_df, test_size=0.5, random_state=42)

val_texts = val_df["text"].astype(str).tolist()
val_labels = val_df[LABEL_COLS].values

In [13]:
deberta_paths = download_wandb_artifacts(DEBERTA_ARTIFACTS, "DeBERTa")
roberta_paths = download_wandb_artifacts(ROBERTA_ARTIFACTS, "RoBERTa")


wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251130_173911-50ah5nf2
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ensemble_deberta
wandb: ⭐️ View project at https://wandb.ai/23f3001910-indian-institute-of-technology-madras/23f3001910-t32025
wandb: 🚀 View run at https://wandb.ai/23f3001910-indian-institute-of-technology-madras/23f3001910-t32025/runs/50ah5nf2
wandb: Downloading large artifact deberta_large_seed_42:v1, 1670.31MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:16.8 (99.3MB/s)
wandb: Downloading large artifact deberta_large_seed_2025:v0, 1670.31MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:23.1 (72.3MB/s)
wandb: Downloading large artifact deberta_large_seed_123:v0, 1670.31MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:19.6 (85.3MB/s)
wandb:                                                                                
wandb: 🚀 View run ensemble_de

In [14]:
deberta_val_probs = get_model_predictions(deberta_paths, val_texts, "DeBERTa")
roberta_val_probs = get_model_predictions(roberta_paths, val_texts, "RoBERTa")


ensemble_val_probs = (
    DEBERTA_WEIGHT * deberta_val_probs +
    ROBERTA_WEIGHT * roberta_val_probs
)

# Find optimal thresholds
optimal_thresholds = find_optimal_thresholds(ensemble_val_probs, val_labels)

Generating DeBERTa Predictions


2025-11-30 17:41:10.648693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764524470.832847      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764524470.890077      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


    Processed 160/1563 texts
    Processed 320/1563 texts
    Processed 480/1563 texts
    Processed 640/1563 texts
    Processed 800/1563 texts
    Processed 960/1563 texts
    Processed 1120/1563 texts
    Processed 1280/1563 texts
    Processed 1440/1563 texts
    Processed 160/1563 texts
    Processed 320/1563 texts
    Processed 480/1563 texts
    Processed 640/1563 texts
    Processed 800/1563 texts
    Processed 960/1563 texts
    Processed 1120/1563 texts
    Processed 1280/1563 texts
    Processed 1440/1563 texts
    Processed 160/1563 texts
    Processed 320/1563 texts
    Processed 480/1563 texts
    Processed 640/1563 texts
    Processed 800/1563 texts
    Processed 960/1563 texts
    Processed 1120/1563 texts
    Processed 1280/1563 texts
    Processed 1440/1563 texts
Generating RoBERTa Predictions
    Processed 160/1563 texts
    Processed 320/1563 texts
    Processed 480/1563 texts
    Processed 640/1563 texts
    Processed 800/1563 texts
    Processed 960/1563 texts
   

In [15]:
val_preds = np.zeros_like(ensemble_val_probs)
for i, label in enumerate(LABEL_COLS):
    val_preds[:, i] = (ensemble_val_probs[:, i] >= optimal_thresholds[label]).astype(int)

val_macro_f1 = f1_score(val_labels, val_preds, average="macro", zero_division=0)
val_per_label_f1 = f1_score(val_labels, val_preds, average=None, zero_division=0)


print(f"Macro F1: {val_macro_f1:.4f}")
for label, f1 in zip(LABEL_COLS, val_per_label_f1):
    print(f"{label:10s}: {f1:.4f}")

Macro F1: 0.8572
anger     : 0.8506
joy       : 0.8593
fear      : 0.8815
surprise  : 0.8499
sadness   : 0.8445


In [16]:
deberta_test_probs = get_model_predictions(deberta_paths, test_texts, "DeBERTa")
roberta_test_probs = get_model_predictions(roberta_paths, test_texts, "RoBERTa")

# Weighted ensemble
ensemble_test_probs = (
    DEBERTA_WEIGHT * deberta_test_probs +
    ROBERTA_WEIGHT * roberta_test_probs
)

# Apply optimal thresholds
final_preds = np.zeros_like(ensemble_test_probs)
for i, label in enumerate(LABEL_COLS):
    final_preds[:, i] = (ensemble_test_probs[:, i] >= optimal_thresholds[label]).astype(int)

# Create submission
for i, label in enumerate(LABEL_COLS):
    test[label] = final_preds[:, i].astype(int)

columns_to_keep = ['id', 'anger', 'fear', 'joy', 'sadness', 'surprise']
test = test[columns_to_keep]

Generating DeBERTa Predictions
    Processed 160/1707 texts
    Processed 320/1707 texts
    Processed 480/1707 texts
    Processed 640/1707 texts
    Processed 800/1707 texts
    Processed 960/1707 texts
    Processed 1120/1707 texts
    Processed 1280/1707 texts
    Processed 1440/1707 texts
    Processed 1600/1707 texts
    Processed 160/1707 texts
    Processed 320/1707 texts
    Processed 480/1707 texts
    Processed 640/1707 texts
    Processed 800/1707 texts
    Processed 960/1707 texts
    Processed 1120/1707 texts
    Processed 1280/1707 texts
    Processed 1440/1707 texts
    Processed 1600/1707 texts
    Processed 160/1707 texts
    Processed 320/1707 texts
    Processed 480/1707 texts
    Processed 640/1707 texts
    Processed 800/1707 texts
    Processed 960/1707 texts
    Processed 1120/1707 texts
    Processed 1280/1707 texts
    Processed 1440/1707 texts
    Processed 1600/1707 texts
Generating RoBERTa Predictions
    Processed 160/1707 texts
    Processed 320/1707 text

In [17]:
test.head()

,id,anger,fear,joy,sadness,surprise
0,0,1,1,0,0,0
1,1,0,0,0,0,0
2,2,1,1,0,0,0
3,3,0,1,0,0,0
4,4,0,1,0,0,1


In [18]:
# Save submission
test.to_csv("submission.csv", index=False)
print("Submission saved")

Submission saved
